# Задание 1. Классификация средствами PyTorch

Используя свой датасет для классификации из курса дисциплины "Машинное обучение" описать и обучить модель для классификации с помощью PyTorch. Выбрать функцию потерь. Использовать объекты TensorDataset и DataLoader.

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

In [4]:
data = pd.read_csv('../data/csgo_task.csv')

In [6]:
data.head()

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,4000.0,0.0,0.0,0.0,5.0,5.0
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,650.0,0.0,0.0,1.0,5.0,5.0
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,10750.0,0.0,0.0,1.0,5.0,5.0


## Обработка

- time_left - длительность раунда
- ct_score - количество очков у команды контр-террористов
- t_score - количество очков у террористов
- map - карта
- bomb_planted - установлена ли бомба
- ct_health и t_health - суммарное здоровье контр-террористов и террористов соответственно
- ct_armor и t_armor - суммарная броня контр-террористов и террористов соответственно
- ct_money и t_money - суммарное количество денег контр-террористов и террористов соответственно
- ct_helmets и t_helmets - суммарное количество шлемов контр-террористов и террористов соответственно
- ct_defuse_kits - количество наборов обезвреживания бомбы у контр-террористо
- ct_players_alive и t_players_alive - количество живых контр-террористов и террористов соответственно

In [7]:
data.drop(['map'], axis=1, inplace=True)

In [8]:
data.dropna(inplace=True)

In [9]:
data.isna().sum()

time_left           0
ct_score            0
t_score             0
bomb_planted        0
ct_health           0
t_health            0
ct_armor            0
t_armor             0
ct_money            0
t_money             0
ct_helmets          0
t_helmets           0
ct_defuse_kits      0
ct_players_alive    0
t_players_alive     0
dtype: int64

In [10]:
data = data.drop_duplicates().reset_index(drop=True)

In [ ]:
columns_to_int = [
    "ct_score",
    "t_score",
    "ct_health",
    "t_health",
    "ct_armor",
    "t_armor",
    "ct_money",
    "t_money",
    "ct_helmets",
    "t_helmets",
    "ct_defuse_kits",
    "ct_players_alive",
    "t_players_alive",
    "bomb_planted",
]

for column in columns_to_int:
    data[column] = data[column].astype(int)

## Создание модели

In [11]:
y, X = data['bomb_planted'].values, data.drop(columns=['bomb_planted']).values
y = y.reshape(-1, 1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((95189, 14), (16799, 14), (95189, 1), (16799, 1))

In [ ]:
train_ds = TensorDataset(torch.from_numpy(X_train).type(torch.float32), torch.from_numpy(y_train).type(torch.float32))
train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)

In [ ]:
x_c, y_c = next(iter(train_dl))
x_c.shape, y_c.shape

In [ ]:
test_ds = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
test_dl = DataLoader(test_ds, batch_size=256, shuffle=True)

In [ ]:
class MyRegressionModel(nn.Module):
    # любая модель в PyTorch - это набор слоев
    # при этом, мы сами определяем порядок их выполнения
    # в конструкторе мы задаем набор слоев с указанием параметров
    def __init__(self):
        super(MyRegressionModel, self).__init__()
        # определяем первый линейный слой (y = wx + b)
        self.first_linear = nn.Linear(40, 120)
        # определяем первый слой ReLU
        self.first_relu = nn.ReLU()
        self.second_linear = nn.Linear(120, 240)
        self.second_relu = nn.ReLU()
        self.third_linear = nn.Linear(240, 60)
        self.third_relu = nn.ReLU()
        self.fourth_linear = nn.Linear(60, 20)
        self.fourth_relu = nn.ReLU()
        self.fifth_linear = nn.Linear(20, 1)

    # в методе forward мы определяем, как слои будут связаны друг с другом
    def forward(self, x):
        # y - результат выполнения первого слоя
        y = self.first_linear(x)
        # в теперь продолжаем накидывать оставшиеся слои
        y = self.first_relu(y)
        y = self.second_linear(y)
        y = self.second_relu(y)
        y = self.third_linear(y)
        y = self.third_relu(y)
        y = self.fourth_linear(y)
        y = self.fourth_relu(y)
        y = self.fifth_linear(y)
        return y

In [ ]:
model = MyRegressionModel()

In [ ]:
loss = nn.MSELoss()
# настраиваем оптимизатор и передаем туда параметры модели
optimizer = torch.optim.Adam(model.parameters(), lr=0.0025)

In [ ]:
epochs = 50
# цикл обучения (по эпохам)
for epoch in range(epochs):
    # за одну эпоху смотрим все батчи (по batch_size элементов)
    for x_b, y_b in train_dl:
        # делаем прямое распространение (получаем предсказание)
        outputs = model(x_b)
        # вычисляем значение функции потерь
        loss_value = loss(outputs, y_b)
        # делаем backward - вычисляются значения .grad у слоев модели
        loss_value.backward()
        # делаем шаг градиентного спуска с заданным у оптимизатора learning_rate
        optimizer.step()
        # зануляем .grad у слоев модели - для нового батча будем акумулировать новый .grad
        optimizer.zero_grad()

    # в конце эпохи выводим значение функции потерь для последнего рассмотренного батча
    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')